In [13]:
import sys
sys.path.append('..')

import numpy as np

from utils.cuda_cluster import *
from utils.dataset import read_data

import core.config as conf


In [3]:
client

Client Scheduler: tcp://127.0.0.1:37381 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 33.47 GB


In [9]:
# data_path = conf.raw_data_path + '*' # for all dataset
data_path = conf.raw_data_path + 'part-00175'
df = read_data(data_path)

number of rows: 3033347


In [11]:
columns = [
    "reply_timestamp",
    "retweet_timestamp",
    "retweet_with_comment_timestamp",
    "like_timestamp"
]
df = df[columns]

In [12]:
df.head()

,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
id,,,,
1,0,0,0,1613048460
2,0,0,0,0
3,1614019431,0,0,0
4,0,0,0,1612780118
5,0,0,0,0


In [16]:
df['is_reply'] = df['reply_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['is_retweet'] = df['retweet_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['is_comment'] = df['retweet_with_comment_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['is_like'] = df['like_timestamp'].compute().applymap(lambda x: 1 if x > 0 else 0).astype(np.int32)

In [17]:
df.head()

,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp,is_reply
id,,,,,
1,0,0,0,1613048460,0
2,0,0,0,0,0
3,1614019431,0,0,0,1
4,0,0,0,1612780118,0
5,0,0,0,0,0
